In [1]:
!wget http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
!bzip2 -dk shape_predictor_68_face_landmarks.dat.bz2

--2024-05-01 06:09:13--  http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
Resolving dlib.net (dlib.net)... 107.180.26.78
Connecting to dlib.net (dlib.net)|107.180.26.78|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64040097 (61M)
Saving to: ‘shape_predictor_68_face_landmarks.dat.bz2’

shape_predictor_68_ 100%[===================>]  61.07M  16.0MB/s    in 3.8s    

2024-05-01 06:09:17 (16.0 MB/s) - ‘shape_predictor_68_face_landmarks.dat.bz2’ saved [64040097/64040097]



In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Sun Apr 21 19:15:43 2024
@author: wxdycq
"""

import cv2
import dlib
import numpy as np
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import Callback
import shutil
import random
import os
import sys

# extract data
isColab = "google.colab" in sys.modules

if isColab:
    from google.colab import drive
    drive.mount("/content/drive", force_remount=True)
    base_path = "/content/drive/My Drive/"
    train_dir = os.path.join(base_path, "train")
    test_dir = os.path.join(base_path, "test 2")


def extract_features(image_path):
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = detector(gray)
    features = {}

    for face in faces:
        landmarks = predictor(gray, face)
        for (start, end, name) in [(36, 42, "left_eye"), (42, 48, "right_eye"), (48, 68, "mouth")]:
            points = np.array([(landmarks.part(n).x, landmarks.part(n).y) for n in range(start, end)])
            x, y, w, h = cv2.boundingRect(points)
            feature_img = img[y:y+h, x:x+w]
            features[name] = Image.fromarray(feature_img)

        return features

# Build CNN model

def build_model(input_shape):
    """Build and compile an enhanced CNN model with advanced architecture features."""
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(input_shape[0], input_shape[1], 1)),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),

        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),

        layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),

        layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),

        layers.Flatten(),
        layers.Dense(512, activation='relu'),
        layers.Dropout(0.4),
        layers.Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

def prepare_image(image):
    image = image.resize((50, 50))
    image_array = np.array(image) / 255.0
    return np.expand_dims(image_array, axis=0)

train_datagen = ImageDataGenerator(
    rescale=1./255, rotation_range=40, width_shift_range=0.2,
    height_shift_range=0.2, shear_range=0.2, zoom_range=0.2,
    horizontal_flip=True, fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)


train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(50, 50),
    batch_size=64,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(50, 50),
    batch_size=64,
    class_mode='categorical'
)



X_train = np.load('X_train.npy')
y_train = np.load('y_train.npy')
X_test = np.load('X_test.npy')
y_test = np.load('y_test.npy')



def reshape_data(X):
    new_shape = np.tile(X[:, :, :1], (1, 1, 2500))
    new_shape = new_shape.reshape(X.shape[0], 50, 50, 3)
    return new_shape

# Model infor, like accuracy, loss
class MetricsCallback(Callback):
    def on_epoch_begin(self, epoch, logs=None):
        print(f"Starting epoch {epoch+1}")

    def on_batch_end(self, batch, logs=None):
        print(f"After batch {batch+1}:")
        print(f" - batch loss: {logs['loss']:.4f}")
        print(f" - batch accuracy: {logs['accuracy']:.4f}")

    def on_epoch_end(self, epoch, logs=None):
        print(f"After epoch {epoch+1}:")
        print(f" - epoch train loss: {logs['loss']:.4f}")
        print(f" - epoch train accuracy: {logs['accuracy']:.4f}")
        if 'val_loss' in logs:
            print(f" - epoch validation loss: {logs['val_loss']:.4f}")
        if 'val_accuracy' in logs:
            print(f" - epoch validation accuracy: {logs['val_accuracy']:.4f}")

# Build and train the model (call function)


model = build_model((X_train.shape[1], X_train.shape[2]))
model.summary()


try:
    model.fit(
        X_train, y_train,
        epochs=10,
        callbacks=[MetricsCallback()],
        validation_data=(X_test, y_test)
    )
except Exception as e:
    print("An error occurred during model training:", e)

# Test model on test data
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")


Streaming output truncated to the last 5000 lines.
 - batch loss: 0.5243
 - batch accuracy: 0.7173
 89/248 [=========>....................] - ETA: 10s - loss: 0.5243 - accuracy: 0.7173After batch 90:
 - batch loss: 0.5236
 - batch accuracy: 0.7177
 90/248 [=========>....................] - ETA: 9s - loss: 0.5236 - accuracy: 0.7177 After batch 91:
 - batch loss: 0.5222
 - batch accuracy: 0.7181
 91/248 [==========>...................] - ETA: 9s - loss: 0.5222 - accuracy: 0.7181After batch 92:
 - batch loss: 0.5233
 - batch accuracy: 0.7177
 92/248 [==========>...................] - ETA: 9s - loss: 0.5233 - accuracy: 0.7177After batch 93:
 - batch loss: 0.5250
 - batch accuracy: 0.7174
 93/248 [==========>...................] - ETA: 9s - loss: 0.5250 - accuracy: 0.7174After batch 94:
 - batch loss: 0.5233
 - batch accuracy: 0.7188
 94/248 [==========>...................] - ETA: 9s - loss: 0.5233 - accuracy: 0.7188After batch 95:
 - batch loss: 0.5236
 - batch accuracy: 0.7184
 95/248 [==